In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from scipy.optimize import fmin_slsqp
import numpy as np
import matplotlib.pyplot as plt
from time import time
import os
import sys
from pathlib import Path
ROOT_PATH = Path().absolute().parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
from frequency_response import FrequencyResponse
from biquad import peaking, digital_coeffs

In [ ]:
fr = FrequencyResponse.read_from_csv(ROOT_PATH.joinpath('results', 'oratory1990', 'harman_over-ear_2018', 'Audeze LCD-4', 'Audeze LCD-4.csv'))

In [150]:
def compare(fp, n):
    fr = FrequencyResponse.read_from_csv(fp)
    t = time()
    fc, q, gain, peq_fr = fr.optimize_peq(n, 48000)
    t = time() - t
    fr.fixed_band_eq = peq_fr
    fr.optimize_parametric_eq(max_filters=n)
    ix = np.sum(fr.frequency < 10e3)
    new_rmse = np.sqrt(np.mean(np.square(fr.equalization[:ix] - fr.fixed_band_eq[:ix])))
    old_rmse = np.sqrt(np.mean(np.square(fr.equalization[:ix] - fr.parametric_eq[:ix])))
    print(f'{fr.name}: RMSE {old_rmse:.2f} -> {new_rmse:.2f}  = {"+" if new_rmse > old_rmse else ""}{new_rmse - old_rmse:.2f} dB ({(new_rmse - old_rmse) / old_rmse * 100:.1f} %) @ {t*1000:.0f} ms')
    #fr.plot_graph(fixed_band_eq_plot_kwargs={'label': 'New optimizer'}, show=True)

In [ ]:
n = [5, 5]
for fp in ROOT_PATH.joinpath('results', 'oratory1990').glob('*/*/*.csv'):
    compare(fp, n)